In [1]:
#!/usr/bin/env python
# coding: utf-8


import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D=True)

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, matthews_corrcoef, average_precision_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import StratifiedKFold
import math

from pandarallel import pandarallel
pandarallel.initialize()

import sys
sys.path.append('/home/ss2686/03_DICTrank')
import argparse
from scripts.evaluation_functions import evaluate_classifier, optimize_threshold_j_statistic


# Path where your data is stored
data_path = '../data/processed_binarised__splits/'

results = {}
held_out_results = []

def get_Mordred_columns_to_use():
    
        datasets = {}
        directory='../data/processed/'
        # Load datasets from given directory
        for foldername in os.listdir(directory):

            if not foldername.startswith('.'):  # Ignore folders starting with a dot

                print(foldername)
                file_path = os.path.join(directory, foldername, f"{foldername}_processed.csv.gz")

                if os.path.exists(file_path):
                    datasets[foldername] = pd.read_csv(file_path, compression='gzip')
                else:
                    print(f"No matching file found for folder: {foldername}")
        
        smiles_list = []
        
        for featuresets in ["sider", "DICTrank"]: 
    
            smiles_list.extend(datasets[featuresets].Standardized_SMILES.to_list())
            print(len(smiles_list))
        smiles_list = list(set(smiles_list))
        
        print(len(smiles_list))
        data = pd.DataFrame(smiles_list, columns=["Standardized_SMILES"])
        
        Ser_Mol_train = data['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_data = calc.pandas(Ser_Mol_train)
        Mordred_table_data = Mordred_table_data.astype('float')
        Mordred_table_data = Mordred_table_data.dropna(axis='columns')
        data_columns = Mordred_table_data.columns
        
        return(data_columns)
    
data_columns = get_Mordred_columns_to_use()


# Assuming PK dataset is regression and others are classification
for dataset in os.listdir(data_path):
    
    # Exclude hidden files or directories like .ipynb_checkpoints
    if dataset.startswith('.'):
        continue
    print(dataset)

    # Get all the file names for this dataset
    all_files = os.listdir(os.path.join(data_path, dataset))

    # Extract activity names by removing the _train.csv.gz or _test.csv.gz from file names
    activity_names = list(set([f.replace("_train.csv.gz", "").replace("_test.csv.gz", "")  for f in all_files if not f.startswith(".ipynb_checkpoints")]))

    for activity in tqdm(activity_names, desc="Processing activities"):
        
        train_path = os.path.join(data_path, dataset, f"{activity}_train.csv.gz")
        test_path = os.path.join(data_path, dataset, f"{activity}_test.csv.gz")

        train_df = pd.read_csv(train_path, compression='gzip')
        test_df = pd.read_csv(test_path, compression='gzip')
        
        Ser_Mol_train = train_df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_train = calc.pandas(Ser_Mol_train)
        Mordred_table_train = Mordred_table_train.astype('float')
               
        Ser_Mol_test = test_df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_test = calc.pandas(Ser_Mol_test)
        Mordred_table_test = Mordred_table_test.astype('float')

        # Retain only those columns in the test dataset
        Mordred_table_train = Mordred_table_train[data_columns]
        Mordred_table_test = Mordred_table_test[data_columns]

        X_train = np.array(Mordred_table_train)
        X_test = np.array(Mordred_table_test)
        y_train = train_df[activity]
        y_test = test_df[activity]

      
        # Classification
        model = RandomForestClassifier(n_jobs=40)
            
        # Hyperparameter Optimization
        param_dist_classification = {'max_depth': randint(10, 20),
                          'max_features': randint(40, 50),
                          'min_samples_leaf': randint(5, 15),
                          'min_samples_split': randint(5, 15),
                          'n_estimators':[200, 300, 400, 500, 600],
                          'bootstrap': [True, False],
                          'oob_score': [False],
                          'random_state': [42],
                          'criterion': ['gini', 'entropy'],
                          'n_jobs': [40],
                          'class_weight' : [None, 'balanced']
                         }
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)   
            
        classification_search = HalvingRandomSearchCV(
                model,
                param_dist_classification,
                factor=3,
                cv=inner_cv,
                random_state=42,
                verbose=1,
                n_jobs=40)
            
        classification_search.fit(X_train, y_train)
        best_model = classification_search.best_estimator_
            
        # Random Over-sampling 
        sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
            
        pipeline = Pipeline(steps=[('sampler', sampler), ('model', best_model)])
        pipeline.fit(X_train, y_train)
            
        # Predict using threshold-optimized model
        predictions_train = pipeline.predict(X_train)
        probs_train = pipeline.predict_proba(X_train)[:, 1]
        probs_test = pipeline.predict_proba(X_test)[:, 1]
            
        # Use the optimize_threshold_j_statistic function to find the best threshold
        best_threshold = optimize_threshold_j_statistic(y_train, probs_train)
        #Apply the best threshold to get binary predictions on the test data
        predictions_test = (probs_test >= best_threshold).astype(int)
            
        # Calculate CV AUC using threshold-optimized model
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, n_jobs=-1, scoring='roc_auc')

        results[activity] = {
                'CV_AUC_mean': np.mean(cv_scores),
                'CV_AUC_std': np.std(cv_scores),
                **evaluate_classifier(y_test, predictions_test, probs_test)
            }
        
        held_out_data = {
            'Dataset': dataset,
            "Actviity": activity,
            'SMILES': test_df['Standardized_SMILES'],
            'True_Value': y_test,
            'Prediction': predictions_test,
            'Probability': probs_test,
            'Best_Threshold': best_threshold
        }
        
        held_out_results.append(pd.DataFrame(held_out_data))         
        # Save results at each step
            
            
    # Save results at each step
    pd.DataFrame(results).T.to_csv('./physicochemical_model_results.csv')
              

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./physicochemical_model_results.csv', index=False)

# Concatenate and save held-out test set results
pd.concat(held_out_results).to_csv('./physicochemical_model_held_out_test_results.csv', index=False)
            


INFO: Pandarallel will run on 76 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
GeneExpressionSMILES
sider
MOA
CellPaintingSMILES
Cmax
DICTrank
1378
2613
1579


  1%|▍                                        | 18/1579 [00:03<05:01,  5.17it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  3%|█▍                                       | 53/1579 [00:07<04:48,  5.29it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...


 47%|████████████████████                       | 42/90 [00:02<00:02, 21.57it/s]
                                                                                
 82%|███████████████████████████████████▎       | 74/90 [00:02<00:00, 55.59it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



100%|███████████████████████████████████████████| 90/90 [00:02<00:00, 31.24it/s]


n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 930
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 46
n_resources: 20
Fitting 5 folds for each of 46 candidates, totalling 230 fits
----------
iter: 1
n_candidates: 16
n_resources: 60
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 6
n_resources: 180
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities: 100%|██████████████████████| 1/1 [01:05<00:00, 65.43s/it]


In [2]:
df = pd.read_csv('./physicochemical_model_held_out_test_results.csv')
df

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold
0,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),O=C(C1CCCCC1)[NH+]1CC(=O)[NH+]2CCc3ccccc3C2C1,1,1,0.866279,0.52507
1,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1,1,1,0.624126,0.52507
2,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),C[NH+](C)CCC=C1c2ccccc2CCc2ccccc21,1,1,0.784482,0.52507
3,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,1,0,0.402885,0.52507
4,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O,1,1,0.734534,0.52507
...,...,...,...,...,...,...,...
488,DICTrank,DICTrank,CCC1(c2ccccc2)C(=O)NCNC1=O,0,1,0.746528,0.59009
489,DICTrank,DICTrank,CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,0,1,0.746718,0.59009
490,DICTrank,DICTrank,CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1,0,1,0.745429,0.59009
491,DICTrank,DICTrank,CCC[NH+](CCC)S(=O)(=O)c1ccc(C(=O)[O-])cc1,0,1,0.776767,0.59009


In [3]:
test_df

,Standardized_SMILES,Standardized_InChI,DICTrank
0,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,InChI=1S/C19H22ClN5O/c20-16-5-3-6-17(15-16)23-...,1
1,CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1,InChI=1S/C15H13FO2/c1-10(15(17)18)12-7-8-13(14...,1
2,C[NH+](C)CCC=C1c2ccccc2CCc2ccccc21,InChI=1S/C20H23N/c1-21(2)15-7-12-20-18-10-5-3-...,1
3,CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,InChI=1S/C26H28Cl2N4O4/c1-19(33)31-10-12-32(13...,1
4,Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O,InChI=1S/C18H19N3O/c1-12-19-9-10-21(12)11-13-7...,1
...,...,...,...
85,CCC1(c2ccccc2)C(=O)NCNC1=O,InChI=1S/C12H14N2O2/c1-2-12(9-6-4-3-5-7-9)10(1...,0
86,CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,InChI=1S/C22H23ClN2O2/c1-2-27-22(26)25-12-9-15...,0
87,CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1,InChI=1S/C12H14N3O2S/c1-3-6-18-8-4-5-9-10(7-8)...,0
88,CCC[NH+](CCC)S(=O)(=O)c1ccc(C(=O)[O-])cc1,"InChI=1S/C13H19NO4S/c1-3-9-14(10-4-2)19(17,18)...",0
